In [21]:
from datasets import load_dataset, Dataset, DatasetDict
import random
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments
from PIL import Image
import requests
import torch
from sentence_transformers import SentenceTransformer, util
import pandas as pd
import numpy as np
#from tqdm import tqdm
from tqdm.notebook import tqdm
import evaluate
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from trl import SFTConfig, SFTTrainer

In [2]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
#os.environ["TORCH_USE_CUDA_DSA"] = "1"

In [3]:
#dataset = load_dataset('code-search-net/code_search_net')
dataset = load_dataset("Fsoft-AIC/the-vault-function", split_set=["test"], trust_remote_code=True)

README.md:   0%|          | 0.00/11.3k [00:00<?, ?B/s]

the-vault-function.py:   0%|          | 0.00/15.0k [00:00<?, ?B/s]

python-00000-of-00001.parquet:   0%|          | 0.00/30.3M [00:00<?, ?B/s]

c-00000-of-00001.parquet:   0%|          | 0.00/25.9M [00:00<?, ?B/s]

c_sharp-00000-of-00001.parquet:   0%|          | 0.00/18.5M [00:00<?, ?B/s]

...

javascript-00000-of-00001.parquet:   0%|          | 0.00/26.4M [00:00<?, ?B/s]

php-00000-of-00001.parquet:   0%|          | 0.00/16.3M [00:00<?, ?B/s]

ruby-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

rust-00000-of-00001.parquet:   0%|          | 0.00/29.0M [00:00<?, ?B/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [4]:
sample_df = pd.DataFrame(dataset['test'])
sample_df.columns

Index(['hexsha', 'repo', 'path', 'license', 'language', 'identifier',
       'return_type', 'original_string', 'original_docstring', 'docstring',
       'docstring_tokens', 'code', 'code_tokens', 'short_docstring',
       'short_docstring_tokens', 'comment', 'parameters', 'docstring_params'],
      dtype='object')

In [5]:
dataset_sample = sample_df.groupby('language', group_keys=False).sample(n=20)
dataset_sample.columns

Index(['hexsha', 'repo', 'path', 'license', 'language', 'identifier',
       'return_type', 'original_string', 'original_docstring', 'docstring',
       'docstring_tokens', 'code', 'code_tokens', 'short_docstring',
       'short_docstring_tokens', 'comment', 'parameters', 'docstring_params'],
      dtype='object')

# Testing the Baseline model

Here we define the system prompt for the Llama 2 model.

In [6]:
def llm_prompt(language, comment, code):
    prompt_gemma = \
    '''You are a helpful agent designed to simplify code documentation for beginner programmers.
    You will be provided with a block of C code and the existing doucmentation that accompanies it.
    Using the provided code as context, give a simplified explanation of the code so that it is understandable
    to beginner programmers. Output absolutely nothing else besides the simplified explanation.
    Make sure to keep any documentation formatting codes present in the simplified explanation.
    If you feel that the existing documentation is simple enough and meaning would be lost by simplifying
    it further, feel free to keep the documentation as is. Here is the original documentation and code:'''
    
    return prompt_gemma

Creating the pipeline for the Gemma 3 model using the HuggingFace transformers library. Modified from the example here: https://huggingface.co/docs/transformers/v4.51.3/en/model_doc/gemma#gemma

In [8]:
# Load model and tokenizer
model_name = "google/gemma-3-4b-it"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float32, device_map="auto")

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.64G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

In [9]:
eval_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [10]:
lan = dataset['test'][0]['language']
code = dataset['test'][0]['original_string']
doc = dataset['test'][0]['original_docstring']
print(f"Original Documentation:\n{dataset['test'][0]['original_docstring']}\n")
print(f"Code:\n{dataset['test'][0]['original_string']}\n")
print("***********************Result************************************")
gemma_raw = llm_prompt(lan, doc, code)

#Tokenize Gemma raw
gemma_inputs = tokenizer(gemma_raw, return_tensors="pt").to(model.device)

# Generate output
with torch.no_grad():
    gemma_outputs = model.generate(**gemma_inputs, max_new_tokens=200, do_sample=True, temperature=0.7)

# Decode and print
print(tokenizer.decode(gemma_outputs[0], skip_special_tokens=True))

Original Documentation:
Build the library mappings tables.

Code:
def build_mapping_tables(app):
    """Build the library mappings tables."""
    env = Environment(loader=FileSystemLoader(f"{DIR_PATH}"))
    template_file = env.get_template("table_template.j2")

    LIST_OF_MAP_DICTS = []
    for attr in dir(lib_mapper):
        if (attr.endswith("MAPPER_REVERSE") or attr.endswith("_MAPPER")) and not (
            attr.startswith("_") or attr.startswith("NETMIKO") or attr.startswith("MAIN")
        ):
            LIST_OF_MAP_DICTS.append(attr)

    for dict_name in LIST_OF_MAP_DICTS:
        lib_name = dict_name.split("_")[0]
        filename = f"{lib_name}_reverse" if "REVERSE" in dict_name else lib_name
        headers = ["NORMALIZED", lib_name] if "REVERSE" in dict_name else [lib_name, "NORMALIZED"]
        rendered_template = template_file.render(lib_names=headers, mappings=getattr(lib_mapper, dict_name))
        with open(f"{DIR_PATH}/netutils/lib_mapping/{filename}_table.rst", "w

In [11]:
generated_documents = []
gemma_semantic_similarities_untrained = []
gemma_metrics_untrained = evaluate.combine(['rouge', 'meteor'])

for instance in tqdm(dataset_sample.itertuples()):
    language = instance.language
    original_doc = instance.original_docstring
    orignal_code = instance.original_string

    message = llm_prompt(language, original_doc, orignal_code)

    #Tokenize Gemma raw
    gemma_inputs = tokenizer(message, return_tensors="pt").to(model.device)

    # Generate output
    with torch.no_grad():
        gemma_outputs = model.generate(**gemma_inputs, max_new_tokens=200, do_sample=True, temperature=0.7)

    # Decode and print
    result = tokenizer.decode(gemma_outputs[0], skip_special_tokens=True)

    #result = result.replace(message, "")

    embedding_original = eval_model.encode(instance.original_docstring, convert_to_tensor=True)
    embedding_predicted = eval_model.encode(result, convert_to_tensor=True)

    gemma_semantic_similarities_untrained.append(util.pytorch_cos_sim(embedding_original, embedding_predicted).item())
    gemma_metrics_untrained.add(predictions=result, references=instance.original_docstring)

[nltk_data] Downloading package wordnet to /home/adeniji/nltk_data...
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/adeniji/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package omw-1.4 to /home/adeniji/nltk_data...


0it [00:00, ?it/s]

In [13]:
print("Untrained Gemma 3: \n")
sims_untrianed = pd.DataFrame(gemma_semantic_similarities_untrained)
sims_untrianed.to_excel('results/Semantic_Similarities_Gemma_untrained.xlsx')
sims_untrianed.describe()

Untrained Gemma 3: 



,0
count,200.000000
mean,0.049620
std,0.087816
min,-0.144458
25%,-0.007388
50%,0.041887
75%,0.101556
max,0.402362


In [14]:
import json
with open('results/rouge_meteor_gemma_untrained.json', 'w') as file:
    mr = gemma_metrics_untrained.compute()
    json.dump(mr, file, indent=4)

# Testing the FineTuned model

Here we Import the training examples from an excel file.

In [16]:
annotated_labeled = pd.read_csv("annotations_labeled.csv")
annotated_labeled.columns

Index(['original_string', 'original_docstring', 'modified_short_docstring',
       'label'],
      dtype='object')

In [37]:
annotated_labeled = annotated_labeled.dropna()
len(annotated_labeled)

160

In [55]:
def training_prompt(documentation, code, modified):
    training_prompt_gemma = \
    '''You are a helpful agent designed to simplify code documentation for beginner programmers. You will be given three 
    variables original_string: \n{original_string} \n\n, original_docstring: \n{original_docstring} \n\n, and modified_docstring: \n{modified_docstring} \n\n. 
    Using these variables you will determine a label value. Using the original_string as context, compare the original_docstring and modified_docstring. 
    These are both code documentation of the original_string. Check if the modified_docstring simplifies the original_docstring while still 
    including important element of the original_string like parameters, return type, and code logic. If so return a 'Yes', otherwise enter a 'No' value'''.format(original_docstring=documentation, original_string=code, modified_docstring=modified)

    return training_prompt_gemma

In [68]:
messages = []
for instance in tqdm(annotated_labeled.itertuples()):
    label = instance.label
    original_doc = instance.original_docstring
    orignal_code = instance.original_string
    modified = instance.modified_short_docstring

    content_user = training_prompt(original_doc, orignal_code, modified)
    content_assitant = str(label)

    #message = {"messages" : [{"content": content_user, "role": "user"}, {"content": content_assitant, "role":"assistant"}]}
    #message = [{"content": content_user, "role": "user"}, {"content": content_assitant, "role":"assistant"}]
    message = {"prompt": content_user, "completion": content_assitant}
    messages.append(message)


0it [00:00, ?it/s]

In [69]:
hf_training_sample = Dataset.from_list(messages)
#hf_training_sample = Dataset.from_list
dataset_dict = DatasetDict({
    "train": hf_training_sample.shuffle(seed=42).select(range(120)),
    "validation": hf_training_sample.shuffle(seed=42).select(range(120, 160))
})
#print(hf_training_sample["messages"][0])"""


In [ ]:
"""hf_training_sample = Dataset.from_pandas(annotated_labeled)

# Tokenize the dataset
def tokenize_function(example):
    return tokenizer(example["original_docstring"], example["modified_short_docstring"], truncation=True, padding="max_length")

tokenized_dataset = hf_training_sample.map(tokenize_function, batched=True)

dataset_dict = DatasetDict({
    "train": tokenized_dataset.shuffle(seed=42).select(range(120)),
    "validation": tokenized_dataset.shuffle(seed=42).select(range(120, 160))
})"""

Map:   0%|          | 0/160 [00:00<?, ? examples/s]

In [70]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds, average="weighted")
    }

training_args = SFTConfig(packing=True)

trainer = SFTTrainer(
    model=model,
    processing_class=tokenizer,
    args=training_args,
    train_dataset=dataset_dict["train"],
    eval_dataset=dataset_dict["validation"],
    compute_metrics=compute_metrics
)

"""trainer = SFTTrainer(
    model=model,
    processing_class=tokenizer,
    args=training_args,
    train_dataset=messages,
    compute_metrics=compute_metrics
)"""

Converting train dataset to ChatML:   0%|          | 0/120 [00:00<?, ? examples/s]

Adding EOS to train dataset:   0%|          | 0/120 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/120 [00:00<?, ? examples/s]

Packing train dataset:   0%|          | 0/120 [00:00<?, ? examples/s]

Converting eval dataset to ChatML:   0%|          | 0/40 [00:00<?, ? examples/s]

Adding EOS to eval dataset:   0%|          | 0/40 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/40 [00:00<?, ? examples/s]

Packing eval dataset:   0%|          | 0/40 [00:00<?, ? examples/s]

'trainer = SFTTrainer(\n    model=model,\n    processing_class=tokenizer,\n    args=training_args,\n    train_dataset=messages,\n    compute_metrics=compute_metrics\n)'

In [43]:
trainer.train()

It is strongly recommended to train Gemma3 models with the `eager` attention implementation instead of `sdpa`. Use `eager` with `AutoModelForCausalLM.from_pretrained('<path-to-checkpoint>', attn_implementation='eager')`.


OutOfMemoryError: CUDA out of memory. Tried to allocate 320.00 MiB. GPU 0 has a total capacity of 47.38 GiB of which 128.44 MiB is free. Process 35041 has 2.76 GiB memory in use. Including non-PyTorch memory, this process has 42.99 GiB memory in use. Of the allocated memory 42.64 GiB is allocated by PyTorch, and 36.07 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
eval_results = trainer.evaluate()
print(eval_results)

In [ ]:
for instance in tqdm(training_sample.itertuples()):
    language = instance.language
    original_doc = instance.original_docstring
    orignal_code = instance.original_string
    mod = instance.modified_short_docstring

    message = training_prompt(language, original_doc, orignal_code, mod)

    pipe_gemma(message, pad_token_id=pipe_gemma.tokenizer.eos_token_id, max_new_tokens=45)

0it [00:00, ?it/s]

In [ ]:
gemma_semantic_similarities_trained = []
gemma_metrics_trained = evaluate.combine(['rouge', 'meteor'])


for instance in tqdm(dataset_sample.itertuples()):
    language = instance.language
    original_doc = instance.original_docstring
    orignal_code = instance.original_string

    message = llm_prompt(language, original_doc, orignal_code)

    #Tokenize Gemma raw
    gemma_inputs = tokenizer(message, return_tensors="pt").to(model.device)

    # Generate output
    with torch.no_grad():
        gemma_outputs = model.generate(**gemma_inputs, max_new_tokens=200, do_sample=True, temperature=0.7)

    # Decode and print
    result = tokenizer.decode(gemma_outputs[0], skip_special_tokens=True)

    #result = result.replace(message, "")

    embedding_original = eval_model.encode(instance.original_docstring, convert_to_tensor=True)
    embedding_predicted = eval_model.encode(result, convert_to_tensor=True)

    gemma_semantic_similarities_trained.append(util.pytorch_cos_sim(embedding_original, embedding_predicted).item())
    gemma_metrics_trained.add(predictions=result, references=instance.original_docstring)

[nltk_data] Downloading package wordnet to /home/adeniji/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/adeniji/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/adeniji/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


0it [00:00, ?it/s]

# Summary statistics results
Semantic Similarity

In [19]:
print("Trained Gemma 3: \n")
sims_trained = pd.DataFrame(gemma_semantic_similarities_trained)
sims_trained.to_excel('results/Semantic_Similarities_Gemma_trained.xlsx')
sims_trained.describe()

Trained Gemma 3: 



,0
count,200.000000
mean,0.504685
std,0.141379
min,0.051231
25%,0.409741
50%,0.516998
75%,0.609724
max,0.840501


ROUGE AND METEOR

In [20]:
with open('results/rouge_meteor_gemma_trained.json', 'w') as file:
    mr = gemma_metrics_trained.compute()
    json.dump(mr, file, indent=4)
